# 토치텍스트 튜토리얼(영어)

Pytorch에서는 텍스트에 대한 여러 추상화 기능을 제공하는 자연어 처리 라이브러리 토치텍스트를 제공한다. 

* File Loading(파일 로드) : 다양한 포맷의 코퍼스를 로드한다.
* Tokenizaition(토큰화) : 문장의 단어 단위로 분리
* Integer encoding(정수 인코딩) : 전체 코퍼스의 단어를 각각의 고유한 정수로 맵핑함.
* Word Vector(단어 벡터): 단어 집합의 단어들에 고유한 임베딩 벡터를 만들어 준다.
* Batching(배치화) : 훈련 샘플들의 배치를 만들어 준다.

## 1. 훈련 데이터와 테스트 데이터로 분리하기

IMDB 리뷰 데이터를 다운로드 받아 이를 훈련 데이터와 테스트 데이터로 분리하여 csv 파일로 저장함.

In [1]:
from torchtext.legacy.data import TabularDataset
import urllib.request
import pandas as pd

In [2]:
# IMDB 리뷰 데이터 다운
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")
df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [3]:
print("전체 샘프의 개수 : {}" .format(len(df)))

전체 샘프의 개수 : 50000


In [5]:
# 훈련데이터와 테스트 데이터를 25000개씩 분리
train_df = df[:25000]
test_df = df[25000:]

In [6]:
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

## 2. 필드 정의하기 (torchtext.data)

* sequential : 시퀀스 데이터 여부
* use_vocab : 단어 집합을 만들 것인지 여부
* tokenize : 어떤 토큰화 함수를 사용할 것인지 지정 (stirng.split이 기본값)
* lower : 영어 데이터를 전부 소문자화한다.
* batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부(False가 기본값)
* is_target : 레이블 데이터 여부 (False가 기본값)
* fix_length : 최대 허용길이

In [31]:
from torchtext.legacy import data # torchtext.data 임포트

# 필드 정의
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  batch_first=True,
                  fix_length=150)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

## 3. 데이터셋 만들기

* path : 파일이 위치한 경로.
* format : 데이터의 포맷.
* fields : 위에서 정의한 필드를 지정. 첫번째 원소는 데이터 셋 내에서 해당 필드를 호칭할 이름, 두번째 원소는 지정할 필드.
* skip_header : 데이터의 첫번째 줄은 무시.

In [32]:
train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

In [33]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))

훈련 샘플의 개수 : 25000
테스트 샘플의 개수 : 25000


In [34]:
print(vars(train_data[0]))

{'text': ['my', 'family', 'and', 'i', 'normally', 'do', 'not', 'watch', 'local', 'movies', 'for', 'the', 'simple', 'reason', 'that', 'they', 'are', 'poorly', 'made,', 'they', 'lack', 'the', 'depth,', 'and', 'just', 'not', 'worth', 'our', 'time.<br', '/><br', '/>the', 'trailer', 'of', '"nasaan', 'ka', 'man"', 'caught', 'my', 'attention,', 'my', 'daughter', 'in', "law's", 'and', "daughter's", 'so', 'we', 'took', 'time', 'out', 'to', 'watch', 'it', 'this', 'afternoon.', 'the', 'movie', 'exceeded', 'our', 'expectations.', 'the', 'cinematography', 'was', 'very', 'good,', 'the', 'story', 'beautiful', 'and', 'the', 'acting', 'awesome.', 'jericho', 'rosales', 'was', 'really', 'very', 'good,', "so's", 'claudine', 'barretto.', 'the', 'fact', 'that', 'i', 'despised', 'diether', 'ocampo', 'proves', 'he', 'was', 'effective', 'at', 'his', 'role.', 'i', 'have', 'never', 'been', 'this', 'touched,', 'moved', 'and', 'affected', 'by', 'a', 'local', 'movie', 'before.', 'imagine', 'a', 'cynic', 'like', 'me

In [35]:
# 필드 구성 확인.
print(train_data.fields.items())

dict_items([('text', <torchtext.legacy.data.field.Field object at 0x7f06d29e3650>), ('label', <torchtext.legacy.data.field.Field object at 0x7f06d29e3110>)])


## 4. 단어 집합(vocabulary) 만들기

토큰화 전처리를 끝냈다면, 이제 각 단어에 고유한 정수를 맵핑해주는 정수 인코딩(Integer enoding) 작업이 필요합니다. 그리고 이 전처리를 위해서는 우선 단어 집합을 만들어준다

정의한 필드에 .build_vocab() 도구를 사용하면 단어 집합을 생성한다.

In [36]:
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

In [37]:
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))

단어 집합의 크기 : 10002


In [38]:
# 단어 집합 내의 단어들을 .stoi를 통해 확인 가능
print(TEXT.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x7f06a4abce90>>, {'<unk>': 0, '<pad>': 1, 'the': 2, 'a': 3, 'and': 4, 'of': 5, 'to': 6, 'is': 7, 'in': 8, 'i': 9, 'this': 10, 'it': 11, 'that': 12, '/><br': 13, 'was': 14, 'as': 15, 'for': 16, 'with': 17, 'but': 18, 'on': 19, 'movie': 20, 'his': 21, 'not': 22, 'are': 23, 'you': 24, 'film': 25, 'have': 26, 'he': 27, 'be': 28, 'at': 29, 'one': 30, 'by': 31, 'an': 32, 'they': 33, 'from': 34, 'who': 35, 'all': 36, 'like': 37, 'so': 38, 'just': 39, 'or': 40, 'has': 41, 'about': 42, "it's": 43, 'if': 44, 'her': 45, 'some': 46, 'out': 47, 'what': 48, 'very': 49, 'when': 50, 'more': 51, 'there': 52, 'even': 53, 'would': 54, 'my': 55, 'good': 56, 'she': 57, 'their': 58, 'only': 59, 'no': 60, 'really': 61, 'can': 62, 'up': 63, 'had': 64, 'which': 65, 'see': 66, 'were': 67, 'than': 68, '-': 69, 'we': 70, 'been': 71, 'into': 72, 'get': 73, 'much': 74, 'will': 75, 'because': 76, 'story': 77, 'most': 78, '

## 5. 토치텍스트의 데이터 로더 만들기

데이터로더는 데이터셋에서 미니 배치만큼 데이터를 로드하게 만들어주는 역할을 하며 토치텍스트에서는 Iterator를 사용하여 데이터로더를 만든디

In [39]:
from torchtext.legacy.data import Iterator

In [40]:
train_loader = Iterator(dataset=train_data, batch_size = 5)
test_loader = Iterator(dataset=test_data, batch_size = 5)

In [41]:
# 훈련 데이터와 테스트 데이터 모두 미니 배치의 수가 5,000개인데 이는 25,000개의 샘플을 배치 크기 5씩 묶어주었기 때문입니다. 
print("훈련 데이터의 미니 배치 수 : {}" .format(len(train_loader)))
print("테스트 데이터의 미니 배치 수: {}" . format(len(test_loader)))

훈련 데이터의 미니 배치 수 : 5000
테스트 데이터의 미니 배치 수: 5000


In [42]:
#첫번쨰 미니배치
batch = next(iter(train_loader)) 

In [43]:
print(type(batch))
print(batch.text)

<class 'torchtext.legacy.data.batch.Batch'>
tensor([[  10,   20,    7,    2,  345,  246,   25,   12,    9,   26,  124,  120,
          392,   82,  102, 1560,    0,   52,    7,  357,   60,  704, 7479,    0,
           40,   92,   80,  587, 2042, 2538,    8, 2432,    3,  480,   10,    0,
            7,  707,   39,  168, 1334,   16,    0, 5192,    6,  131,  137,   21,
            0,  602,  481,    2,    0,  239,   27,   61,    0,    9,  296, 1059,
           55,   84,  148,   10,  138,    2,  119,  184,   54,   26,    6,   28,
            2,  344, 5581,   44,    9,   67,    0,   40,    0,    9,   54,   26,
          642,   55, 4129,  137,    4, 1261,  116,   29,    2,    0, 2572,    8,
           58,  245,  429,  446,    0, 5192,    2,  667,    6,  585,    8,    3,
          873,  149,    7,  512,   19,    2,  162, 1758,  902,   15,  468,    8,
         3608,   86,   22,  409,  110,   84,  148,   10,  150,   11,   91,  171,
          863,   28,    2,   59,  169,   24,    0,    1,    1,   

In [44]:
batch.text[0]

tensor([  10,   20,    7,    2,  345,  246,   25,   12,    9,   26,  124,  120,
         392,   82,  102, 1560,    0,   52,    7,  357,   60,  704, 7479,    0,
          40,   92,   80,  587, 2042, 2538,    8, 2432,    3,  480,   10,    0,
           7,  707,   39,  168, 1334,   16,    0, 5192,    6,  131,  137,   21,
           0,  602,  481,    2,    0,  239,   27,   61,    0,    9,  296, 1059,
          55,   84,  148,   10,  138,    2,  119,  184,   54,   26,    6,   28,
           2,  344, 5581,   44,    9,   67,    0,   40,    0,    9,   54,   26,
         642,   55, 4129,  137,    4, 1261,  116,   29,    2,    0, 2572,    8,
          58,  245,  429,  446,    0, 5192,    2,  667,    6,  585,    8,    3,
         873,  149,    7,  512,   19,    2,  162, 1758,  902,   15,  468,    8,
        3608,   86,   22,  409,  110,   84,  148,   10,  150,   11,   91,  171,
         863,   28,    2,   59,  169,   24,    0,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,   